#                                                   Hull Tactical Market Prediction Challenge                                                        

In this competition, we build a machine learning model to predict future market returns based on historical financial data.
Our goal is to design a trading strategy that maximizes profit while minimizing risk measured using an **Adjusted Sharpe Ratio**.

The model learns patterns from past data (training set) and predicts investment positions (in the test set), simulating how an investor might allocate money between a risk-free asset and the market.
The better the model balances profit and stability, the higher its Sharpe score and leaderboard rank.

# 1️⃣ Importing Required Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

NumPy → Efficient numerical computation.

Pandas → Data loading, analysis, and manipulation.

os → To explore the dataset directory provided by Kaggle.

# 📊 3️⃣ Loading the Dataset

In [ ]:
import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/train.csv')
test = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/test.csv')

In [ ]:
train.head(5)

In [ ]:
test.head(5)

We suppress warnings for cleaner output.

train.csv → Historical daily market data with returns (the target).

test.csv → Mock test data that mimics the unseen evaluation set structure.

Display shapes and previews to verify columns and understand data scale.

# 4️⃣ Importing Core Evaluation Tools

In [ ]:
import polars as pl
from scipy.optimize import minimize, Bounds
import kaggle_evaluation.default_inference_server

Polars: Similar to Pandas but faster : required for the evaluation API.

SciPy’s minimize: Used for optimization of predictions.

Kaggle evaluation API: Required for submissions — ensures no future data leakage.

# 5️⃣ Defining Constants

In [ ]:
MAX_POSITION = 2.0
MIN_POSITION = 0.0
ANNUAL_DAYS = 252   # typical trading days per year
EPS = 1e-12         # small value to avoid division by zero


The position (portfolio allocation) must stay between 0 (no exposure) and 2 (double leverage).

The annualization factor (252) helps compute annualized Sharpe ratio.

EPS prevents division errors in numerical calculations.

# 6️⃣ Custom Error for User-visible Exceptions

In [ ]:
class UserVisibleError(Exception):
    """Custom error type for messages displayed to the user."""
    pass

Used by Kaggle’s evaluation API, if a submission predicts invalid values (outside 0–2 range), this error provides clear feedback.

# 7️⃣ Custom Adjusted Sharpe Ratio Function

In [ ]:
def adjusted_sharpe(solution: pd.DataFrame, submission: pd.DataFrame) -> float:
    """
    Computes the adjusted Sharpe ratio used in this competition.
    """
    solution = solution.copy()
    solution['position'] = submission['prediction'].astype(float)

    # Validate limits
    if solution['position'].max() > MAX_POSITION + 1e-9:
        raise UserVisibleError(f"Prediction above max limit {MAX_POSITION}")
    if solution['position'].min() < MIN_POSITION - 1e-9:
        raise UserVisibleError(f"Prediction below min limit {MIN_POSITION}")

    # Strategy returns
    solution['strategy_returns'] = (
        solution['risk_free_rate'] * (1 - solution['position']) +
        solution['position'] * solution['forward_returns']
    )

    # Excess returns (strategy - risk-free)
    excess = solution['strategy_returns'] - solution['risk_free_rate']

    # Geometric mean of returns (annualized)
    cum_excess = float((1.0 + excess).prod())
    mean_excess = cum_excess ** (1.0 / len(solution)) - 1.0 if cum_excess > 0 else -1.0

    # Volatility
    std_excess = float(solution['strategy_returns'].std())
    if std_excess == 0:
        raise ZeroDivisionError("Zero strategy std; Sharpe undefined.")

    # Base Sharpe ratio
    sharpe = mean_excess / (std_excess + EPS) * np.sqrt(ANNUAL_DAYS)
    strat_vol = float(std_excess * np.sqrt(ANNUAL_DAYS) * 100.0)

    # Market baseline metrics
    market_excess = solution['forward_returns'] - solution['risk_free_rate']
    market_cum = float((1.0 + market_excess).prod())
    market_mean = market_cum ** (1.0 / len(solution)) - 1.0 if market_cum > 0 else -1.0
    market_std = float(solution['forward_returns'].std())
    market_vol = float(market_std * np.sqrt(ANNUAL_DAYS) * 100.0)

    # Penalties for volatility or underperformance
    excess_vol_penalty = (1.0 + max(0.0, strat_vol / (market_vol + EPS) - 1.2)) if market_vol > 0 else 1.0
    return_gap = max(0.0, (market_mean - mean_excess) * 100.0 * ANNUAL_DAYS)
    return_penalty = 1.0 + (return_gap ** 2) / 100.0

    # Adjusted score
    score = sharpe / (excess_vol_penalty * return_penalty + EPS)
    return float(min(score, 1_000_000.0))


This is the core metric.
It computes a volatility-penalized Sharpe ratio:

Rewards high returns relative to volatility.

Penalizes excessive volatility or low returns vs market.

Ensures scores stay within a safe numeric range.

# 8️⃣ Load Train Data for Optimization

In [ ]:
train = pd.read_csv(
    "/kaggle/input/hull-tactical-market-prediction/train.csv",
    index_col="date_id"
)

for col in ["forward_returns", "risk_free_rate"]:
    if col not in train.columns:
        raise RuntimeError(f"Missing column in train: {col}")


Reloads train.csv with date_id as index (useful for alignment).

Verifies critical columns exist (for computing the metric).

# 9️⃣ Select Recent Data for Model Optimization

In [ ]:
N_DAYS = 180
if len(train) < N_DAYS:
    N_DAYS = len(train)

recent = train.iloc[-N_DAYS:].copy()


We only use the most recent 180 days to simulate how the model would perform on current market conditions, mirroring the competition’s rolling forecast nature.

# 🔟 Define Optimization Objective

In [ ]:
def objective(x):
    positions = np.clip(np.asarray(x, dtype=float), MIN_POSITION, MAX_POSITION)
    submission = pd.DataFrame({'prediction': positions}, index=recent.index)
    return -adjusted_sharpe(recent, submission)


The goal is to maximize the Sharpe ratio.
Since scipy.optimize.minimize() performs minimization, we minimize the negative Sharpe ratio.

# 1️⃣1️⃣ Optimize Portfolio Allocation

In [ ]:
x0 = np.full(N_DAYS, 0.05, dtype=float)   # initial guess
bounds = Bounds(MIN_POSITION, MAX_POSITION)

res = minimize(
    objective,
    x0,
    method="Powell",
    bounds=bounds,
    tol=1e-8,
    options={"maxiter": 2000, "xtol": 1e-8, "ftol": 1e-8, "disp": True}
)

print("Optimization result:", res)

# Final predictions
optimal_preds = np.clip(res.x if res.success else x0, MIN_POSITION, MAX_POSITION).astype(np.float64)


We start with a constant small exposure (0.05).

Use Powell optimization (gradient-free, robust for noisy objectives).

The result gives daily optimal positions that maximize the adjusted Sharpe ratio.

# 1️⃣2️⃣ Define the Predict Function (for Submission)

In [ ]:
counter = 0

def predict(batch: pl.DataFrame) -> float:
    global counter, optimal_preds
    if counter < len(optimal_preds):
        value = optimal_preds[counter]
    else:
        value = optimal_preds[-1]
    print(f"[{counter}] Prediction: {float(value):.8f}")
    counter += 1
    return float(value)


The evaluation API calls predict() one row (day) at a time.

The function returns the next optimized portfolio position.

Once all precomputed positions are used, it repeats the last one.

# 1️⃣3️⃣ Run the Evaluation Server

In [ ]:
server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    server.serve()
else:
    server.run_local_gateway(("/kaggle/input/hull-tactical-market-prediction/",))


This block connects your predict() function to the Kaggle evaluation system.

In a Kaggle environment:

server.serve() runs during submission.

server.run_local_gateway() simulates local evaluation in the notebook.